In [8]:
import pandas
import openpyxl #leer y escribir en excel 
import sqlalchemy, pymysql #conectar python con la base de datos Mysql

# ETL

In [24]:
import pandas as pd
import numpy as np
import os 
import glob

In [29]:
MAPA_TR = {
    "01_VOLTAJE_AC_DEL_SISTEMA_L1_L2": "VOLTAJE_AC_L1_l2",
    "02_VOLTAJE_AC_DEL_SISTEMA_L2-L3": "VOLTAJE_AC_L2_l3",
    "03_VOLTAJE_AC_DEL_SISTEMA_L3-L1": "VOLTAJE_AC_L3_l1",
    "11_CORRIENTE_AC_DE_LA_CARGA_L1": "CORRIENTE_AC_L1",
    "12_CORRIENTE_AC_DE_LA_CARGA_L2": "CORRIENTE_AC_L2",
    "13_CORRIENTE_AC_DE_LA_CARGA_L3": "CORRIENTE_AC_L3",
    "14_POTENCIA_ACTIVA_DE_LA_CARGA": "POTENCIA_ACTIVA_KW",
    "15_POTENCIA_REACTIVA_DE_LA_CARGA": "POTENCIA_REACTIVA_KVAR",
    "16_POTENCIA_APARENTE_DE_LA_CARGA": "POTENCIA_APARENTE_KVA",
    "17_FACTOR_DE_POTENCIA_DE_LA_CARGA": "FACTOR_DE_POTENCIA"
}

MAPA_ML = {
    "ANALOG_INPUT_-_ML_CURRENT_AC_R": "CORRIENTE_AC_R",
    "ANALOG_INPUT_-_ML_CURRENT_AC_S": "CORRIENTE_AC_S",
    "ANALOG_INPUT_-_ML_CURRENT_AC_T": "CORRIENTE_AC_T",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_R-S": "VOLTAJE_AC_RS",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_S-T": "VOLTAJE_AC_ST",
    "ANALOG_INPUT_-_ML_VOLTAGE_AC_T-R": "VOLTAJE_AC_TR",
    "ANALOG_INPUT_C_SALA_S01": "TEMPERATURA_SALA_S01",
    "ANALOG_INPUT_C_SALA_S02": "TEMPERATURA_SALA_S02"
}

MAPA_RECT1 = {
    "01_-_VOLTAJE_AC_DEL_SISTEMA" : "VOLTAJE_AC_VS",
    "02_-_VOLTAJE_DC_DEL_SISTEMA": "VOLTAJE_DC_VS",
    "03_-_CORRIENTE_DC_DEL_SISTEMA": "CORRIENTE_DC_CS",
    "11_-_CORRIENTE_DC_DE_LA_CARGA": "CORRIENTE_DC_CARGA"
}

MAPA_RECT2 = {
    "01_-_VOLTAJE_AC_DEL_SISTEMA" : "VOLTAJE_AC_VS",
    "02_-_VOLTAJE_DC_DEL_SISTEMA": "VOLTAJE_DC_VS",
    "03_-_CORRIENTE_DC_DEL_SISTEMA": "CORRIENTE_DC_CS",
    "11_-_CORRIENTE_DC_DE_LA_CARGA": "CORRIENTE_DC_CARGA"
}

In [30]:
def limpiar_fecha(fecha_str):
    """
    TR - IDEO CALI,"01 - VOLTAJE AC DEL SISTEMA L1-L2","216.0","V","1/1/2025, 12:50:03 a.Â m."

    Arregla el formato y elimina caracteres de codificación corruptos (Â).
    Entrada: '1/1/2025, 12:00:00 a.Â m.' o '1/1/2025, 12:00:00 a.\xa0m.'
    Salida: '1/1/2025 12:00:00 AM'
    """

    if not isinstance(fecha_str, str):
        return str(fecha_str)
    

    limpia = fecha_str.replace("Â", "") \
                      .replace("\xa0", " ") \
                      .replace(",", "") 
    
    limpia = limpia.lower() \
                   .replace("a. m.", "AM") \
                   .replace("p. m.", "PM") \
                   .replace("a.m.", "AM") \
                   .replace("p.m.", "PM") \
                   .replace("am", "AM") \
                   .replace("pm", "PM")
    
    return limpia.strip() # Eliminar espacios 

In [31]:
fecha= "12:50:03 a.Â m."
fecha1 = limpiar_fecha(fecha)
print(fecha1)

12:50:03 AM


In [ ]:
def procesar_carpeta_sensor(ruta, nombre_carpeta, nombre_columna_db):
    """
    Lee los 12 meses de una carpeta, limpia y consolida.
    """

    ruta_busqueda = os.path.join(ruta, nombre_carpeta, "*.csv") # O *.cvs si tienen error de extension
    archivos = glob.glob(ruta_busqueda)
    
    if not archivos:
        print(f"ADVERTENCIA: No se encontraron archivos en {nombre_carpeta}")
        return None

    print(f"Procesando {nombre_carpeta} ({len(archivos)} archivos)...")
    
    dfs_meses = []
    
    for archivo in archivos:
        try:
            # Leemos el CSV. Importante: dtype=str para no romper la lectura con 'Unplugged'
            df_temp = pd.read_csv(archivo, encoding='latin-1', dtype=str) # Probamos latin-1 por los caracteres raros
            
            df_temp['Time_Clean'] = df_temp['Time'].apply(limpiar_fecha)
            df_temp['timestamp'] = pd.to_datetime(df_temp['Time_Clean'], format='%d/%m/%Y %I:%M:%S %p', errors='coerce')
            
            # Convertimos Value a números. Los textos se vuelven NaN.
            df_temp['valor_numerico'] = pd.to_numeric(df_temp['Value'], errors='coerce')
            
            #Eliminar filas con NaN
            df_temp = df_temp.dropna(subset=['valor_numerico', 'timestamp'])
            
            # Seleccionamos solo lo que sirve (ya limpio)
            df_util = df_temp[['timestamp', 'valor_numerico']].copy()
            
            dfs_meses.append(df_util)
            
        except Exception as e:
            print(f"Error leyendo {os.path.basename(archivo)}: {e}")

    if not dfs_meses:
        return None

    # Unimos Enero-Diciembre verticalmente
    df_anual = pd.concat(dfs_meses, ignore_index=True)
    
    # Ordenamos por fecha
    df_anual = df_anual.sort_values('timestamp')
    df_anual = df_anual.set_index('timestamp')
    

    # ********************************************************************************************RESAMPLING (Normalización de tiempo)
    # Como los datos vienen a 12:01:59 y otros a 12:05:00, promediamos cada 10 minutos
    # Esto alinea los datos y reduce el ruido.
    df_resampled = df_anual.resample('10min').mean()
    
    # Renombramos la columna 'valor_numerico' al nombre real del sensor (ej: voltaje_ac_l1_l2)
    df_resampled.columns = [nombre_columna_db]
    
    return df_resampled

In [35]:
ELEMENTOS_DCE = [
    ("TR",    "./DCE_DATOS_2025/TR",    MAPA_TR),
    ("ML",    "./DCE_DATOS_2025/ML",    MAPA_ML),
    ("Rect1", "./DCE_DATOS_2025/Rect1", MAPA_RECT1),
    ("Rect2", "./DCE_DATOS_2025/Rect2", MAPA_RECT2)
]

if __name__ == "__main__":
    print("ETL")

    # Iteramos sobre cada dispositivo
    for nombre_dev, ruta_base, mapa_sensores in ELEMENTOS_DCE:
        print(f"\n" + "="*50)
        print(f"PROCESANDO DISPOSITIVO: {nombre_dev}")
        print(f"="*50)
        
        dfs_del_dispositivo = []
        
        # Procesamos cada carpeta de sensor de este dispositivo
        for carpeta_sensor, columna_db in mapa_sensores.items():
            # LLAMAMOS A TU FUNCIÓN (Asegúrate de pasarle 'ruta_base' si la modificas para aceptar argumento)
            # NOTA: Tu función 'procesar_carpeta_sensor' actualmente usa una variable global RUTA_BASE_TR.
            # DEBES MODIFICAR la función para que reciba la ruta como parámetro.
            
            df_sensor = procesar_carpeta_sensor(ruta_base, carpeta_sensor, columna_db)
            
            if df_sensor is not None:
                dfs_del_dispositivo.append(df_sensor)
        
        if dfs_del_dispositivo:
            print(f"Unificando sensores de {nombre_dev}...")
            df_final = pd.concat(dfs_del_dispositivo, axis=1)
            
            # Limpieza final (interpolación o reset index)
            df_final = df_final.reset_index()
            
            # Guardar
            archivo_salida = f"consolidado_{nombre_dev}_2025.csv"
            df_final.to_csv(archivo_salida, index=False)
            print(f"Guardado {archivo_salida} ({len(df_final)} filas)")
        else:
            print(f"ADVERTENCIA: No se generaron datos para {nombre_dev}")

    print("\n PROCESO TERMINADO.")

ETL

PROCESANDO DISPOSITIVO: TR
Procesando 01_VOLTAJE_AC_DEL_SISTEMA_L1_L2 (12 archivos)...
Procesando 02_VOLTAJE_AC_DEL_SISTEMA_L2-L3 (12 archivos)...
Procesando 03_VOLTAJE_AC_DEL_SISTEMA_L3-L1 (12 archivos)...
Procesando 11_CORRIENTE_AC_DE_LA_CARGA_L1 (12 archivos)...
Procesando 12_CORRIENTE_AC_DE_LA_CARGA_L2 (12 archivos)...
Procesando 13_CORRIENTE_AC_DE_LA_CARGA_L3 (12 archivos)...
Procesando 14_POTENCIA_ACTIVA_DE_LA_CARGA (12 archivos)...
Procesando 15_POTENCIA_REACTIVA_DE_LA_CARGA (12 archivos)...
Procesando 16_POTENCIA_APARENTE_DE_LA_CARGA (12 archivos)...
Procesando 17_FACTOR_DE_POTENCIA_DE_LA_CARGA (12 archivos)...
   ⚡ Unificando sensores de TR...
Guardado consolidado_TR_2025.csv (52560 filas)

PROCESANDO DISPOSITIVO: ML
Procesando ANALOG_INPUT_-_ML_CURRENT_AC_R (12 archivos)...
Procesando ANALOG_INPUT_-_ML_CURRENT_AC_S (12 archivos)...
Procesando ANALOG_INPUT_-_ML_CURRENT_AC_T (12 archivos)...
Procesando ANALOG_INPUT_-_ML_VOLTAGE_AC_R-S (12 archivos)...
Procesando ANALOG_INP